# Pairs bootstrap and correlation

[Dataset download](https://s3.amazonaws.com/bebi103.caltech.edu/data/bee_sperm.csv)

<hr>

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

In [2]:
import numpy as np
import pandas as pd

import numba

import bebi103

import bokeh
bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

We continue our analysis of the [drone sperm quality data set](https://s3.amazonaws.com/bebi103.caltech.edu/data/bee_sperm.csv). Let's load it and remind ourselves of the content.

In [3]:
df = pd.read_csv(os.path.join(data_path, "bee_sperm.csv"), comment='#')
df.head()

,Specimen,Treatment,Environment,TreatmentNCSS,Sample ID,Colony,Cage,Sample,Sperm Volume per 500 ul,Quantity,ViabilityRaw (%),Quality,Age (d),Infertil,AliveSperm,Quantity Millions,Alive Sperm Millions,Dead Sperm Millions
0,227,Control,Cage,1,C2-1-1,2,1,1,2150000,2150000,96.7263814616756,96.726381,14,0,2079617,2.1500,2.079617,0.070383
1,228,Control,Cage,1,C2-1-2,2,1,2,2287500,2287500,96.3498079760595,96.349808,14,0,2204001,2.2875,2.204001,0.083499
2,229,Control,Cage,1,C2-1-3,2,1,3,87500,87500,98.75,98.750000,14,0,86406,0.0875,0.086406,0.001094
3,230,Control,Cage,1,C2-1-4,2,1,4,1875000,1875000,93.2874208336941,93.287421,14,0,1749139,1.8750,1.749139,0.125861
4,231,Control,Cage,1,C2-1-5,2,1,5,1587500,1587500,97.7925061050061,97.792506,14,0,1552456,1.5875,1.552456,0.035044


## Correlation

We might wish to investigate how two measured quantities are correlated. For example, if the number of dead sperm and the number of alive sperm are closely correlated, this would mean that a given drone produces some quantity of sperm and some fraction tend to be dead. Let's take a look at this.

In [4]:
# Set up plot on log scale
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=300,
    x_axis_label="alive sperm (millions)",
    y_axis_label="dead sperm (millions)",
    x_axis_type="log",
    y_axis_type="log",
)

# Only use values greater than zero for log scale
inds = (df["Alive Sperm Millions"] > 0) & (df["Dead Sperm Millions"] > 0)

# Populate glyphs
for color, (treatment, g) in zip(
    bokeh.palettes.Category10_3, df.loc[inds, :].groupby("Treatment")
):
    p.circle(
        source=g,
        x="Alive Sperm Millions",
        y="Dead Sperm Millions",
        color=color,
        legend_label=treatment,
    )

p.legend.location = "bottom_right"

bokeh.io.show(p)

There seems to be some correlation (on a log scale), but it is difficult to tell. We can compute the correlation with the **bivariate correlation coefficient**, also known as the Pearson correlation. It is the plug-in estimate of the correlation between variables (in this case alive and dead sperm). The **correlation** is the covariance divided by the geometric mean of the individual variances

The bivariate correlation coefficient is implemented with `np.corrcoef()`, but we will code our own and JIT it for speed.

In [5]:
@numba.njit
def bivariate_r(x, y):
    """
    Compute plug-in estimate for the bivariate correlation coefficient.
    """
    return (
        np.sum((x - np.mean(x)) * (y - np.mean(y)))
        / np.std(x)
        / np.std(y)
        / np.sqrt(len(x))
        / np.sqrt(len(y))
    )

We can use it to compute the bivariate correlation coefficient for the logarithm of alive and dead sperm.

In [6]:
bivariate_r(
    np.log(df.loc[inds, "Alive Sperm Millions"].values),
    np.log(df.loc[inds, "Dead Sperm Millions"].values),
)

0.5219944217488051

## Pairs bootstrap confidence intervals

How can we get a confidence interval on a correlation coefficient? We can again apply the bootstrap, but this time, the replicate is a *pair* of data, in this case a dead sperm count/alive sperm count pair. The process of drawing pairs of data points from an experiment and then computing bootstrap replicates from them is called **pairs bootstrap**. Let's code it up for this example with the bivariate correlation.

Our strategy in coding up the pairs bootstrap is to draw bootstrap samples of the *indices* of measurement and use those indices to select the pairs.

In [7]:
@numba.njit
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=len(data))


@numba.njit
def draw_bs_pairs(x, y):
    """Draw a pairs bootstrap sample."""
    inds = np.arange(len(x))
    bs_inds = draw_bs_sample(inds)
    
    return x[bs_inds], y[bs_inds]

With our pairs sampling function in place, we can write a function to compute replicates.

In [8]:
@numba.njit
def draw_bs_pairs_reps_bivariate(x, y, size=1):
    """
    Draw bootstrap pairs replicates.
    """
    out = np.empty(size)

    for i in range(size):
        out[i] = bivariate_r(*draw_bs_pairs(x, y))

    return out

Finally, we can put it all together to compute confidence intervals on the correlation. To start, we extract all of the relevant measurements as Numpy arrays to allow for faster resampling (and that's what our Numba'd functions require).

In [9]:
# Extract NumPy arrays (only use values greater than zero for logs)
inds = (df["Alive Sperm Millions"] > 0) & (df["Dead Sperm Millions"] > 0)

alive_ctrl = df.loc[
    (inds) & (df["Treatment"] == "Control"), "Alive Sperm Millions"
].values

alive_pest = df.loc[
    (inds) & (df["Treatment"] == "Pesticide"), "Alive Sperm Millions"
].values

dead_ctrl = df.loc[
    (inds) & (df["Treatment"] == "Control"), "Dead Sperm Millions"
].values

dead_pest = df.loc[
    (inds) & (df["Treatment"] == "Pesticide"), "Dead Sperm Millions"
].values

Now we can compute the bootstrap replicates using our `draw_bs_pairs_reps_bivariate()` function.

In [10]:
# Get reps
bs_reps_ctrl = draw_bs_pairs_reps_bivariate(
    np.log(alive_ctrl), np.log(dead_ctrl), size=10000
)

bs_reps_pest = draw_bs_pairs_reps_bivariate(
    np.log(alive_pest), np.log(dead_pest), size=10000
)

And from the replicates, we can compute and print the 95% confidence interval.

In [11]:
# Get the confidence intervals
conf_int_ctrl = np.percentile(bs_reps_ctrl, [2.5, 97.5])
conf_int_pest = np.percentile(bs_reps_pest, [2.5, 97.5])

# Plot confidence intervals
summaries = [
    dict(
        label="control",
        estimate=bivariate_r(np.log(alive_ctrl), np.log(dead_ctrl)),
        conf_int=conf_int_ctrl,
    ),
    dict(
        label="treated",
        estimate=bivariate_r(np.log(alive_pest), np.log(dead_pest)),
        conf_int=conf_int_pest,
    ),
]

bokeh.io.show(
    bebi103.viz.confints(summaries, x_axis_label="bivariate correlation of logs")
)

We see a clear correlation in both samples, with a wide, but positive, confidence interval. Note that we did this analysis on a log scale, since the data span several orders of magnitude.

## Computing environment

In [12]:
%load_ext watermark
%watermark -v -p numpy,pandas,numba,bokeh,bebi103,jupyterlab

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

numpy     : 1.21.5
pandas    : 1.4.3
numba     : 0.55.1
bokeh     : 2.4.3
bebi103   : 0.1.11
jupyterlab: 3.4.4

